** INTRODUCTION ** 

We have been writing a paper on climate change impacts on snowpack and fire risk in the western United States using simulations from the Integrated Scenarios project. The domain for the paper includes five mountain ranges (Cascades, Sierra Nevada, Northern Rockies, Southern Rockies, White Mountains) and six lowland regions (Missouri Basin, Coastal North, Coastal South, Lower Colorado, Great Basin, and Northwest Interior). To understand climate impacts on fire and to tie this into changes in snowpack, we have been looking at dead fuel moisture. Our goal is to compute 100-hr and 1000-hr time lags of dead fuel moisture (DFM) and to look at changes in DFM between the 2020s and 2080s. 

** ASSUMPTIONS ** 

These are necessary because available archived data from Integrated Scenarios includes maximum and minimum temperature, specific humidity, and precipitation (and not relative humidity). 

1. RH. Deriving RH from specific humidity (approximating the mixing ratio w as q). 
2. Climate class. The NFDRS method uses climate class to approximate a precipitation rate, which is 0.25 in/hr for climate classes 1 and 2 and 0.05 in/hr for climate classes 3 and 4. The amount of variability in daily DFM is very dependent on this. The precip rate is then used to determine the precip duration, which cannot exceed 8 hours if this method is used. 
3. SATVPD. Using saturation vapor pressure evaluated at the average daily temperature rather than observed temperature (dry-bulb temperature of afternoon observation time). 
4. AMBVP. Using average RH (derived from q) rather than observed RH. 
5. EMC. Using Tmin for dew point temperature and RH (derived from q). 

** DATA INPUTS AND ASSUMPTIONS APPLIED ** 

In [2]:
import pandas as pd
df = pd.read_pickle('df_dfm_algorithm')
cols = ['Data Inputs Needed','Source for my Algorithm','Assumptions Made for my Algorithm']
df[cols]

,Data Inputs Needed,Source for my Algorithm,Assumptions Made for my Algorithm
0,Tmin,Input,none
1,Tmax,Input,none
2,RHobs,derived from q,#1. (specific humidity = mixing ratio)
3,RHmin,derived,uses AMBVP estimated using RH instead of RHobs
4,RHmax,derived,uses AMBVP estimated using RH instead of RHobs
5,Tdew,Tmin,uses Tmin for Tdew
6,elevation,taken from VIC soil parameter file,none
7,pressure,estimated,none
8,ambient vapor pressure (AMBVP),estimated,#4. uses estimated RH instead of RHobs input
9,saturation vapor pressure (SATVPD),estimated,#3. (Tavg instead of Tdew) and uses T0 = 273.15 K


** METHOD **

The method used here is outlined in the National Fire Danger Rating system: basic equations article. It can be found here: http://www.fs.fed.us/psw/publications/documents/psw_gtr082/psw_gtr082.pdf
Citation: Cohen, JD and Deeming JE. 1985. The National Fire Danger Rating System: basic equations. Pacific Southwest Forest and Range Experiment Station, Berkeley, CA. 

Essentially the approach draws on regression equations developed by Simard (1968) on the basis of tables in the Wood Handbook (US Forest Products Laboratory 1955, revised 1974). EMC is expressed as a percent moisture content. 

The citation for Simard (1968) is: Simard AJ (1968). The moisture content of forest fuels I: A review of basic concepts. Ottawa, ONT: Forest Fire Research Institute, 1968. Rep. FF-X-14, 47p. 




** EQUATIONS ** 

$$ e_s(T) = 611 * \exp( \frac {17.67 * (T - T_0)} { T - 29.65} ) $$ where $T_0$ = 273.15 K 
$$ p(h) = 101325 * (1 - (2.25577 * 10^-5) * h)^{5.25588} $$ where h = elevation (in meters)
$$ AMBVP = \frac {RH * e_{s_{min}}} {100} $$ where AMBVP = ambient vapor pressure  
$$ SATVPD = e_s $$ where SATVPD = saturation vapor pressure for $T_{dew}$ 
$$ SATVPN = e_s (T_{min}) $$ where SATVPN = saturation vapor pressure for $T_{min}$
$$ SATVPX = e_s (T_{max}) $$ where SATVPx = saturation vapor pressure for $T_{max}$
$$ RH = 100.0 * \frac {w} {w_s} $$ where $$ w_s = 0.622 * \frac {e_s} {p} $$ and w = q (approximating the mixing ratio as the specific humidity) 
$$ RH_{max} = 100.0 * \frac {AMBVP} {SATVPN} $$
$$ RH_{min} = 100.0 * \frac {AMBVP} {SATVPX} $$

$$ PPTDUR(PPTAMT,WETRAT) = \frac {PPTAMT} {WETRAT} + 0.49 $$ where PPTDUR = duration of precipitation (in hours), WETRAT = precipitation rate (in in/hr) and PPTAMT = amount of precipitation (in inches). PPTDUR is rounded to the nearest integer. 

if RH < 10, then 
$$ EMC(RH,T) = 0.03229 + (0.281073 * RH) - (0.000578 * T * RH) $$

if RH $\ge$ 10 and RH $\le$ 50, then 

$$ EMC(RH,T) = 2.22749 + (0.160107*RH) - (0.014784*T) $$ 

if RH > 50, then

$$ EMC(RH,T) = 21.0606 + (0.005565*(RH^2)) - (0.00035*RH*T) - (0.483199*RH) $$ where T is in Fahrenheit 

$$ EMC_{min} = EMC(RH_{min},T_{max}) $$ 
$$ EMC = EMC(RH,T_{min}) $$ 
$$ EMC_{max} = EMC(RH_{max},T_{min}) $$

$$ PHI = lat * 0.01745 $$ where lat is in degrees
$$ DECL = 0.41008 * \sin((julian - 82) * 0.01745) $$ where julian is the Julian day of the year
$$ DAYLIT = 24 * (1.0 - \frac{\arccos(\tan(PHI)*\tan(DECL))}{3.1416}) $$ where DAYLIT = number of hours between sunrise and sunset
$$ EMC_{bar} = \frac {(DAYLIT * EMC_{min}) + (24 - DAYLIT) * EMC_{max} )} {24.0} $$ where $EMC_{bar}$ is the weighted 24-hour average EMC
$$ BNDRYH = \frac {(24.0 - PPTDUR) * EMC_{bar} + PPTDUR * (0.5 * PPTDUR + 41.0)} {24.0} $$ where BNDRYH = weighted 24-hour average boundary condition 
$$ MC100 = YMC100 + (BNDRYH - YMC100) * (1.0 - 0.87 * \exp(-0.24)) $$ where $ YMC100 = 5.0 + (5.0 * CLIMAT) $ to initialize a computational period and is otherwise the MC100 value calculated the previous day (and CLIMAT is the climate class, e.g. 1,2,3, or 4) 


** MY APPROACH ** 

To understand how the algorithm for 100-hr DFM was performing, I imported 100-hr DFM data available through UI Metdata to compare with my algorithm for DFM. I compared for one gridcell located in the WA Cascades, with slightly different resolutions between the UI Metdata DFM (4 km) and Integrated Scenarios (6 km). I coded this up in Python (Ipython notebook available for reference) and did the comparison for one water year (1982). 

Initially, I compared results between the UI 100-hr DFM data and my algorithm for DFM using input data from Integrated Scenarios. 

![100-hr DFM1](dfm_comparison_uidfm_is.png) 

The correlation (r value) is 0.39. 

Since there was a low correlation between UI Metdata DFM and my 100-hr DFM algorithm (using the NFDRS approach with IS inputs), I attempted to isolate the differences by downloading UI Metdata inputs and using those instead of the IS data inputs. 

![100-hr DFM2](dfm_comparison_uidfm_nfdrsui.png)

This increased the correlation by a large amount, but still low, with r = 0.65.

For the comparison above, I used the same UI Metdata inputs as I was using from IS, Tmax, Tmin, q (specific humidity), and Precip. To further isolate potential issues with deriving RH from q, I then tried using RHmax and RHmin from UI Metdata as inputs instead of deriving RH from q. 

![100-hr DFM3](dfm_comparison_uidfm_nfdrsui_RH.png) 

This improved the correlation further, with r = 0.73. The difference in correlation between this and the one above (using derived RH) points to possible issues with Assumption 1. 

All of these results are shown together in one figure below. NFDRS/ represents my algorithm, UI Metdata DFM represents 100-hr DFM data from UI. 

![100-hr DFM4](dfm_comparison_one_gridcell.png) 

** ISSUES TO FLAG ** 
1. Variability is very sensitive to WETRAT (precip rate in in/hr). This is dependent on the climate class. Although variability is smaller for the climate class used here (2), the correlations are higher. For example, if the climate class is changed to class 3, and precip rate is 0.05 in/hr (instead of 0.25 in/hr as used here), the variability changes dramatically (see figure below). ![100-hr DFM6](dfm_comparison_one_gridcell_climclass3.png) However, the correlations are much lower; the maximum correlation from above (UI Metdata DFM compared with my algorithm using UI Metdata inputs with RH) is only 0.60. 
2. The correlation improves when UI Metdata Inputs with RHmin and RHmax are used. This highlights possible problems with my estimation of RH. Additionally, there are times when RH > 100 in my algorithm, which obviously doesn't make sense. 
3. EMC values for my algorithm do seem a bit high based on the literature, but this might not be unreasonable. 